# Structure gemini output in a robust way

In [1]:
from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="""
    List a few asian soups recipes, a yummy description,
    and list the ingredients.
    """
)
print(response.text)

Here are a few delicious Asian soup recipes, each with a mouth-watering description and a list of key ingredients!

---

### 1. Tom Yum Goong (Thai Hot & Sour Shrimp Soup)

*   **Yummy Description:** Brace yourself for a vibrant explosion of flavors! This iconic Thai soup is a fiery, tangy, and aromatic masterpiece that brightens your palate with every spoonful. Imagine plump, succulent shrimp swimming in a fragrant broth infused with lemongrass, galangal, kaffir lime leaves, and a perfect balance of spicy chilies and sour lime juice. It's exhilarating, refreshing, and utterly addictive.

*   **Key Ingredients:**
    *   **Broth Base:** Chicken or vegetable stock
    *   **Protein:** Fresh shrimp (peeled and deveined)
    *   **Aromatics:** Lemongrass (bruised), Galangal (sliced), Kaffir lime leaves (torn), Bird's eye chilies (sliced or crushed)
    *   **Vegetables:** Mushrooms (straw, oyster, or button), Cherry tomatoes
    *   **Flavorings:** Fish sauce, Fresh lime juice, Tom Yum pa

In [2]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents="""
    List a few asian soups recipes, a yummy description,
    and list the ingredients, structured as a JSON fomat. 
    """
)
print(response.text)

Here are a few delicious Asian soup recipes presented in JSON format, complete with yummy descriptions and ingredient lists!

```json
[
  {
    "name": "Tonkotsu Ramen",
    "cuisine": "Japanese",
    "description": "A truly soul-warming Japanese noodle soup featuring an incredibly rich and creamy, milky-white pork bone broth that has been simmered for hours until deeply savory. It's traditionally served with springy ramen noodles, tender slices of chashu pork, a perfectly marinated soft-boiled egg (ajitsuke tamago), crisp nori seaweed, and a sprinkle of fresh scallions. Each spoonful is a journey into umami bliss!",
    "ingredients": [
      "Pork bone broth (homemade or high-quality store-bought)",
      "Fresh ramen noodles",
      "Chashu pork (braised pork belly slices)",
      "Ajitsuke Tamago (marinated soft-boiled egg)",
      "Nori seaweed sheets",
      "Scallions (chopped, for garnish)",
      "Wood ear mushrooms (kikurage, rehydrated and sliced)",
      "Beni shoga (red pi

In [5]:
from pydantic import BaseModel
from google import genai

class Recipe(BaseModel):
    recipe_name: str
    description: str
    ingredients: list[str]

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="""
    List a few asian soups recipes, a yummy description,
    and list the ingredients
    """,
    config={"response_mime_type": "application/json", "response_schema": list[Recipe]},
)
print(response.text)



[{"recipe_name":"Tom Yum Goong","description":"A vibrant and aromatic Thai hot and sour soup, famous for its bold flavors and spicy kick. It features succulent shrimp, mushrooms, and a fragrant broth infused with lemongrass, galangal, kaffir lime leaves, and chili.","ingredients":["Shrimp","Mushrooms","Lemongrass","Galangal","Kaffir lime leaves","Bird's eye chilies","Fish sauce","Lime juice","Cherry tomatoes","Cilantro","Chicken or shrimp broth"]},
{"recipe_name":"Miso Soup","description":"A classic Japanese comfort food, this savory and light soup is made with dashi (a flavorful broth typically from bonito flakes and kombu) and miso paste, garnished with silken tofu, seaweed, and green onions.","ingredients":["Miso paste","Dashi broth","Tofu (silken)","Wakame seaweed","Green onions"]},
{"recipe_name":"Vietnamese Pho Bo","description":"A deeply aromatic and complex Vietnamese beef noodle soup, celebrated for its rich, clear broth simmered for hours with spices like star anise, cinnamon

In [8]:
recipes = response.parsed
type(recipes), len(recipes), recipes[0]

(list,
 3,
 Recipe(recipe_name='Tom Yum Goong', description='A vibrant and aromatic Thai hot and sour soup, famous for its bold flavors and spicy kick. It features succulent shrimp, mushrooms, and a fragrant broth infused with lemongrass, galangal, kaffir lime leaves, and chili.', ingredients=['Shrimp', 'Mushrooms', 'Lemongrass', 'Galangal', 'Kaffir lime leaves', "Bird's eye chilies", 'Fish sauce', 'Lime juice', 'Cherry tomatoes', 'Cilantro', 'Chicken or shrimp broth']))

In [9]:
recipes[-1].recipe_name, recipes[-1].ingredients

('Vietnamese Pho Bo',
 ['Beef bones',
  'Beef chuck or brisket',
  'Rice noodles',
  'Onions',
  'Ginger',
  'Star anise',
  'Cinnamon sticks',
  'Cloves',
  'Cardamom pods',
  'Fish sauce',
  'Sugar',
  'Bean sprouts',
  'Lime wedges',
  'Jalapeño slices',
  'Basil',
  'Cilantro'])

# Simulate house price

In [10]:
from typing import Literal

class Home(BaseModel):
    price: int
    rent: int
    living_area: int
    munber_rooms: float
    adress: str
    type: Literal["apartment", "house"]

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="""
    List 50 apartments and houses in Sweden with their monthly_fee, their price, living area, 
    number of rooms, address, type if it is apartment or house. 
    All currencies are in SEK.
    """,
    config={"response_mime_type": "application/json", "response_schema": list[Home]},
)

homes = response.parsed
homes[:3]

[Home(price=3800000, rent=4500, living_area=65, munber_rooms=2.5, adress='Storgatan 12A, 111 20 Stockholm', type='apartment'),
 Home(price=6200000, rent=3200, living_area=140, munber_rooms=5.0, adress='Ekallén 5, 222 30 Lund', type='house'),
 Home(price=2950000, rent=3800, living_area=50, munber_rooms=2.0, adress='Vasagatan 7B, 411 10 Göteborg', type='apartment')]

In [18]:
import pandas as pd

df = pd.DataFrame([dict(home) for home in homes])
df.head()

,price,rent,living_area,munber_rooms,adress,type
0,3800000,4500,65,2.5,"Storgatan 12A, 111 20 Stockholm",apartment
1,6200000,3200,140,5.0,"Ekallén 5, 222 30 Lund",house
2,2950000,3800,50,2.0,"Vasagatan 7B, 411 10 Göteborg",apartment
3,7800000,4800,180,6.0,"Björkvägen 15, 753 25 Uppsala",house
4,1950000,2900,35,1.5,"Drottninggatan 33, 111 50 Stockholm",apartment


In [19]:
df.query("price < 2_000_000")

,price,rent,living_area,munber_rooms,adress,type
4,1950000,2900,35,1.5,"Drottninggatan 33, 111 50 Stockholm",apartment
18,1800000,2800,30,1.0,"Ringvägen 100, 118 60 Stockholm",apartment
38,1750000,2700,28,1.0,"Hornsgatan 130, 117 28 Stockholm",apartment
